### Overview
This notebook loads the data sets that are available on Kaggle see [data](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data) and adjusts it to our need. Part 1 generates the data that is used in the notebook LSTM.ipynb and VAR.ipynb. Part 2 generates the data that is used in the notebook LSTM_mav.ipynb (for running Part 2 of this notebook Part 1 has to be executed before).

In [1]:
import pandas as pd
import numpy as np

### Part 1

In [2]:
#load Kaggle data
oil = pd.read_csv('datasets/oil.csv')
holidays = pd.read_csv('datasets/holidays_events.csv')
stores = pd.read_csv('datasets/stores.csv')
train = pd.read_csv('datasets/train.csv')
transactions = pd.read_csv('datasets/transactions.csv')
test = pd.read_csv('datasets/test.csv')

In [3]:
#change dtype of date column to datetime
oil['date'] = oil['date'].apply(pd.to_datetime)
holidays['date'] = holidays['date'].apply(pd.to_datetime)
train['date'] = train['date'].apply(pd.to_datetime)
transactions['date'] = transactions['date'].apply(pd.to_datetime)
test['date'] = test['date'].apply(pd.to_datetime)

### Train set

In [6]:
oil = oil.set_index('date').asfreq('D').reset_index()
oil['dcoilwtico'] = oil['dcoilwtico'].interpolate('linear').ffill().bfill()
train = train.merge(oil)
train = train.rename(columns={"dcoilwtico": "oilprice"})


#change family names to numeric values
fam = np.unique(train['family'])
fam_lookup = pd.DataFrame(data = {'family': fam, 'family_id': range(len(fam))})
train  = train.merge(fam_lookup)


#split up date into multiple informations
train['day'] = train['date'].apply(lambda time: time.day)
train['month'] = train['date'].apply(lambda time: time.month)
train['weekday'] = train['date'].apply(lambda time: time.dayofweek)
train['year'] = train['date'].apply(lambda time: time.year)

In [9]:
#holiday handling
def isholiday(row):
  #data
    date = row['date']
    event = holidays.loc[holidays['date'] == date][0:1]
    id = row['store_nbr']
    city = stores.loc[stores['store_nbr'] == id]['city'].values
    state = stores.loc[stores['store_nbr'] == id]['state'].values
  
  
    #check if events apply:
    if len(event) > 0:
        national = event['locale'].values == 'National'
        regional = event['locale'].values == 'Regional' and event['locale_name'].values == state
        local = event['locale'].values == 'Local' and event['locale_name'].values == city

        if national[0] or regional[0] or local[0]:
            if event['type'].values == 'Holiday' and  event['transferred'].values == False:
                return 2
            elif event['type'].values == 'Transfer':
                return 2
            elif event['type'].values == 'Bridge':
                return 1
            elif event['type'].values == 'Work Day':
                return 0

    #otherwise: check if weekend
    if row['weekday']< 5:
        return 0
    else: 
        return 1


#lookup table  (adds dates to stores table and gets holiday type)
date1, date2 = train['date'].min(), train['date'].max()
holiday_lookup = stores[['store_nbr', 'type']].copy()
holiday_lookup.loc[:,'holiday'] = 0

holiday_lookup = pd.merge(holiday_lookup, pd.DataFrame({'date': pd.date_range(date1, date2, freq = 'd')}), how = "cross")
holiday_lookup['weekday'] = holiday_lookup['date'].apply(lambda time: time.dayofweek)
holiday_lookup.loc[:, 'holiday'] = holiday_lookup.apply(lambda row: isholiday(row), axis = 1)
holiday_lookup = holiday_lookup.drop('weekday', axis = 1)


#join with train
train = train.merge(holiday_lookup)

In [22]:
train = train.drop('oilprice', axis=1) #drop because oilprice is containted in two columns
train = train.merge(oil)
train = train.rename(columns={'dcoilwtico':'oilprice'})

In [29]:
train.to_csv('datasets/train_big.csv')

### Test set

In [35]:
test = pd.read_csv('datasets/test.csv')
test['date'] = test['date'].apply(pd.to_datetime)

# preprocess test data
test = test.merge(oil)
test = test.rename(columns={"dcoilwtico": "oilprice"})



#change family names to numeric values
test  = test.merge(fam_lookup)


#split up date into multiple informations
test['day'] = test['date'].apply(lambda time: time.day)
test['month'] = test['date'].apply(lambda time: time.month)
test['weekday'] = test['date'].apply(lambda time: time.dayofweek)
test['year'] = test['date'].apply(lambda time: time.year)

In [38]:
date1, date2 = test['date'].min(), test['date'].max()
holiday_lookup = stores[['store_nbr', 'type']].copy()
holiday_lookup.loc[:,'holiday'] = 0

holiday_lookup = pd.merge(holiday_lookup, pd.DataFrame({'date': pd.date_range(date1, date2, freq = 'd')}), how = "cross")
holiday_lookup['weekday'] = holiday_lookup['date'].apply(lambda time: time.dayofweek)
holiday_lookup.loc[:, 'holiday'] = holiday_lookup.apply(lambda row: isholiday(row), axis = 1)
holiday_lookup = holiday_lookup.drop('weekday', axis = 1)


#join with test
test = test.merge(holiday_lookup)

In [45]:
#store data 
test.to_csv('datasets/test_big.csv')

### Part 2
Motivation: We include an additional feature column to our data set namley the moving average (MAV) this feature should 'guide' the LSTM to the right mean. For later prediction in the test set we dont have the MAV so we extrapolate it using SARIMAX.

In [2]:
#load data that was generated in Part 1
train_big = pd.read_csv('datasets/train_big.csv')
test_big = pd.read_csv('datasets/test_big.csv')

Include MAV in the training data as additional column.

In [ ]:
recompute_train_MAV = False #choose if MAV for the training data set should be recomputed 

if recompute_train_MAV:
    #incude MAV column in dataframe train_big
    stores = list(train_big['store_nbr'].unique())
    families = list(train_big['family_id'].unique())
    train_big_MAV = train_big.copy()
    train_big_MAV['MAV'] = 0
    av = 7 #sliding window
    for store in stores:
        for family in families:
            df =  train_big_MAV.loc[(train_big_MAV['store_nbr']==store)&(train_big_MAV['family_id']==family)].copy()
            df['MAV'] = df['sales'].rolling(av).mean()
            df['MAV'] = df['MAV'].replace(np.nan,df['MAV'].iloc[av])
            train_big_MAV['MAV'].loc[(train_big_MAV['store_nbr']==store)&(train_big_MAV['family_id']==family)] = df['MAV'].values

    #store df
    train_big_MAV.to_csv('datasets/train_big_MAV.csv')

Extrapolate the MAV from the training data to put it as an additional column into the test data.

In [ ]:
import statsmodels.api as sm

recompute_test_MAV = True #choose if MAV for the test data set should be recomputed 
train_big_MAV = pd.read_csv('datasets/train_big_MAV.csv')
if recompute_test_MAV:
    #incude MAV column in dataframe train_big
    stores = list(test_big['store_nbr'].unique())
    families = list(test_big['family_id'].unique())
    test_big_MAV = test_big.copy()
    test_big_MAV['MAV'] = 0
    av = 7 #sliding window
    for store in stores:
        for family in families:
            #select MAV values from the past to train SARIMAX model
            df_MAV = train_big_MAV['MAV'].loc[(train_big_MAV['store_nbr']==store)&(train_big_MAV['family_id']==family)]
            df_MAV = pd.DataFrame(df_MAV.loc['2017-01-01':]) #select 2017-01-01 as starting date for the SARIMAX model (just for efficiency resons)
            df_MAV.index = pd.DatetimeIndex(df_MAV.index)
            model=sm.tsa.statespace.SARIMAX(df_MAV,order=(20, 1, 1)) #this order tuple seems to be best after checking a few
            results=model.fit()
            forecast = results.predict(start=len(df_MAV['MAV']),end=len(df_MAV['MAV'])+15,dynamic=True) #predict the 16 dates that are missing in the test set
            #include the prediction into the test set
            test_big_MAV['MAV'].loc[(test_big_MAV['store_nbr']==store)&(test_big_MAV['family_id']==family)] = forecast.values

    #store df
    test_big_MAV.to_csv('datasets/test_big_MAV.csv')